In [18]:
import numpy as np
import math

import european_call as ec

In [19]:
# 4. For the case of Geometric Brownian Motion and a European call option
# with parameters, r = 0.05, sigma = 0.2, T = 1, S_0 = 100, K = 100, investigate
# the following forms of variance reduction:

r = 0.05
sigma = 0.2
S_0 = 100
K = 100
T = 1

N = 1000

analytic_value = ec.european_call(r, sigma, T, S_0, K, 'value')

print 'Analytic Value: ', analytic_value

Analytic Value:  10.450583572185565


In [20]:
# payoff function
def f_W(r, sigma, T, S_0, K, W):
    W_T = [x*math.sqrt(T) for x in W]
    S_T = [S_0*math.exp((r - sigma**2/2)*T + sigma*w) for w in W_T]
    f_S = [math.exp(-r*T)*max(s-K, 0) for s in S_T]
    return f_S

In [38]:
# MC Estimation

f = []
for i in range(0,N):
    W = np.random.standard_normal(N)
    payoff = f_W(r, sigma, T, S_0, K, W)
    f.append(payoff)

MC_variance = np.var(f)

print 'Monte Carlo Variance: ', MC_variance

Monte Carlo Variance:  216.86176749420693


In [39]:
# 4. a) First, try antithetic variables using (f(W) + f(-W))/2 where W is the 
# value of the underlying Brownian motion at maturity

W = np.random.standard_normal(10**6)

f_pos = f_W(r, sigma, T, S_0, K, W)
f_neg = f_W(r, sigma, T, S_0, K, -1*W)
f = [0.5*(f_pos[i] + f_neg[i]) for i in range(0,10**6)]

estimated_value = np.mean(f)
anti_variance = np.var(f)
correlation = np.corrcoef(f_pos, f_neg)[0][1]
var_reduction = 100*(1 - 0.5*(1 + correlation))

print 'Estimated Call Option Value: ', estimated_value

print 'Antithetic Variables Variance: ', anti_variance

print 'Correlation: ', correlation

print 'Variance reduction: ', var_reduction, '%'

# What is the estimated correlation between f(W) and f(-W)? How much variance 
# reduction does this give? 

# There is some variance reduction. It is not extremely substantial.

Estimated Call Option Value:  10.43727014658852
Antithetic Variables Variance:  53.90218949565772
Correlation:  -0.5010800649713179
Variance reduction:  75.05400324856589 %


In [40]:
# 4. b) Second, try using exp(-rT)S(T) as a control variate, noting that its 
# expected value is S(0). Again, how much variance reduction does this give?

# Control Variate
def g_W(r, sigma, T, S_0, K, W):
    W_T = [x*math.sqrt(T) for x in W]
    S_T = [S_0*math.exp((r - sigma**2/2)*T + sigma*w) for w in W_T]        
    g_S = [math.exp(-r*T)*S for S in S_T]
    return g_S

In [46]:
W = np.random.standard_normal(10**6)

g = g_W(r, sigma, T, S_0, K, W)
f = f_W(r, sigma, T, S_0, K, W)
    
covariance = np.cov(f,g)[0][1]
min_val = covariance/np.var(g)

f_hat = f - min_val*(np.array(g) - S_0)

variance = np.var(f_hat)
estimated_value = np.mean(f_hat) 
correlation = np.corrcoef(f, g)[0][1]
var_reduction = 100*(1 - (1 - correlation**2))

print 'Estimated Call Option Value: ', estimated_value

print 'Control Variates Variance: ', variance

print 'Correlation: ', correlation

print 'Variance reduction: ', var_reduction, '%'


Estimated Call Option Value:  10.44646557386335
Control Variates Variance:  31.529817089103567
Correlation:  0.9244757917425168
Variance reduction:  85.46554895179533 %
